---
title: Theoretical formulas for space plasma physics
subtitle: for tplot
---

default units are SI units for space plasma physics

- B : magnetic field, nT
- n : number density, cm^-3
- T : temperature, eV

Reference

- [plasmapy.formulary](https://docs.plasmapy.org/en/stable/formulary/index.html) provides theoretical formulas for calculation of physical quantities helpful for plasma physics.

In [ ]:
# | default_exp ds.tplot.formulary
# | export
import pytplot
from xarray import DataArray
from space_analysis.plasma.formulary.numpy import np_Alfven_speed
from space_analysis.ds.tplot import store_data
from loguru import logger

In [ ]:
# | export
def ts_Alfven_speed(
    B: DataArray,  # magnetic field in the plasma, could be a component, as plasmapy will take `abs` of it
    density: DataArray,  # particle density of the plasma
    name="Alfven_speed",
    time="time",
    **kwargs,
):
    if not B.coords[time].equals(density.coords[time]):
        logger.info(
            "Time of B and density are not the same, interpolating density to B"
        )
        density = density.interp_like(B)

    Alfven_speed = np_Alfven_speed(B=B.to_numpy(), density=density.to_numpy(), **kwargs)

    return DataArray(
        Alfven_speed, coords={time: B.coords[time]}, dims=[time], name=name
    )


def tplot_Alfven_speed(
    B: str,  # magnetic field in the plasma, could be a component, as plasmapy will take `abs` of it
    density: str,  # particle density of the plasma
    **kwargs,
):
    B_da: DataArray = pytplot.data_quants[B]
    n_da: DataArray = pytplot.data_quants[density]

    Alfven_speed = ts_Alfven_speed(B=B_da, density=n_da, **kwargs)

    return store_data(Alfven_speed)